In [15]:
import webbrowser
import urllib.parse
from dotenv import load_dotenv
from http.server import BaseHTTPRequestHandler, HTTPServer
import os
import requests
import base64
import json

In [2]:
load_dotenv()
client_id = os.getenv("client_id")
redirect_uri = "http://127.0.0.1:8888/callback/"
scopes = "playlist-modify-private playlist-modify-public"


In [3]:
auth_url = "https://accounts.spotify.com/authorize?" + urllib.parse.urlencode({
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": redirect_uri,
    "scope": scopes
})
auth_code = None
class CallbackHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        global auth_code
        query = urllib.parse.urlparse(self.path).query
        params = urllib.parse.parse_qs(query)
        if "code" in params:
            auth_code = params["code"][0]
            print("Authorization code:", auth_code)
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"You can close this window now.")
            # save the code somewhere or trigger token exchange
        else:
            self.send_response(400)
            self.end_headers()

server = HTTPServer(("127.0.0.1", 8888), CallbackHandler)
print("Waiting for Spotify redirect...")
webbrowser.open(auth_url)
print("Go to the URL above and log in.")
server.handle_request()
server.server_close()

Waiting for Spotify redirect...
Go to the URL above and log in.
Opening in existing browser session.
Authorization code: AQAKneBk7tqFeZArVXIkGfvZzLQeTPmkoyo1ZgPBc1TSymX95_UwhBCpKhlPDsIC2N-5_OO7Kf1r55hmIj65fFQhAJF62OyXuGOHeqXFO-n2oRkeAsgs-uvbu5TtzRwTxc_2IflL3rPj6K5VeUzb4_wD5sbnd5D5bu3gAIqVORPqWPnjpIoBYZ8Hs-rDE3CogI-Cf1ekta8LNbORUI1eNdnzSmn5PZZBOyni27h8-LRcq2BL


127.0.0.1 - - [03/Sep/2025 14:43:45] "GET /callback/?code=AQAKneBk7tqFeZArVXIkGfvZzLQeTPmkoyo1ZgPBc1TSymX95_UwhBCpKhlPDsIC2N-5_OO7Kf1r55hmIj65fFQhAJF62OyXuGOHeqXFO-n2oRkeAsgs-uvbu5TtzRwTxc_2IflL3rPj6K5VeUzb4_wD5sbnd5D5bu3gAIqVORPqWPnjpIoBYZ8Hs-rDE3CogI-Cf1ekta8LNbORUI1eNdnzSmn5PZZBOyni27h8-LRcq2BL HTTP/1.1" 200 -


In [10]:
print(auth_code)

AQBN1uOndWASkxJilPcmaktZAs78HrPJwQrAHaXunPMvqeqhKteL_I1SwEG8Y5RUiZ1YtKEukiQSn6bPW7jZbUZM7mlhek4aQcVwy4IVkV9XAiGsSDCCGvijVfEus2nWrTk7pum3BmFm9nDja0AjuQ7vcrLn09wgt5M1Lep4W2YvTgkB3HDkt7r8DjqrnxuMpyKufN_nXzjfCTdiwaGTS4hd2e137JNaBiYUoehSCLuYA3Vj


In [4]:
client_secret = os.getenv("client_secret")

In [5]:
def get_tokens(auth_code):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(
            f"{client_id}:{client_secret}".encode()
        ).decode()
    }
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri,
    }

    r = requests.post(url, headers=headers, data=data)
    r.raise_for_status()
    return r.json()

In [6]:
r = get_tokens(auth_code)

In [7]:
print(r)

{'access_token': 'BQDIH0jRmpe_nS3ty6DjJfp3r5s9Nbv2z6HKeEHi7L9QD5LmUiECNSt3Rw4rZ7hUXdMzcD2EoRs1iEV7m7C267Edz9_6_T86CgQJY_wrx-dAe2bizoWd6Gn6LvPzx7dd3RNcR5NeMU1LRlkI9dN-VvzlJPUboEh9OtjE6b4JX8VEbNZwLcpomhVhcZXZB8o5htd-460viZ2ydBSjQFnjMu1hJn31C3DFsqWjOy_pT82vdYN4oXmeIgA5foNkd5p9stzIdYwo5JRG-mtsqNF5tcsi88-sndqEm7fIfdttTc8', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQB6_H04XZoHsfmN1FGd0chHc810YGWV7ULB09cFdsg1O07dlrQuNqvpG2-mfx8shaGvaEkPGf29ySYNKSRkCpEj_RNq5zzOahn_4aQX8XCor1mQWYwBrynTUkgvqQXXy04', 'scope': 'playlist-modify-private playlist-modify-public'}


In [8]:
def refresh_access_token(refresh_token):
    url = "https://accounts.spotify.com/api/token"
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {"Authorization": f"Basic {auth_header}"}
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token
    }

    r = requests.post(url, headers=headers, data=data)
    r.raise_for_status()
    return r.json()

In [11]:
refresh_token = os.getenv("refresh_token")
tokens = refresh_access_token(refresh_token)
print(tokens)

{'access_token': 'BQC7CzrtgVSnXhpfevISuPUywTJ8r2ilXwIWt4jM9NBxWm8iL4iYosjNlfL8mplUk-ooB6G1QgKaVE0GrBK7CzXFqkudYqTS-0WsW_Km6rzZCtC3AIf8e48i53f49JdwV0amUoxJlxVETbpVI2qPwzSbrIGXcOr6QlzPMc-WVRwwp9XAP8IYR1g0r_MBP0L5VGYLPcOVlPDRdFWhdsfeLqlayK_WgQPkfNRAqWPTyr54Mr7fpwg7SwbOxQvTwhySr83IF_7wXf-b22zi0mUGvaDSaeznuXldHUpxMov4BbE', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'playlist-modify-private playlist-modify-public'}


In [12]:
access_token = refresh_access_token(refresh_token)['access_token']

In [13]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
r = requests.get("https://api.spotify.com/v1/me", headers=headers)
r.raise_for_status()
user_id = r.json()["id"]

In [ ]:
playlist_data = {
    "name": "My API Playlist",
    "description": "Created via Python CLI",
    "public": False
}

r = requests.post(f"https://api.spotify.com/v1/users/{user_id}/playlists",
                  headers=headers, data=json.dumps(playlist_data))
r.raise_for_status()
playlist = r.json()

In [17]:
r

<Response [201]>